In [2908]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2909]:
# default_exp fastrl.fastai.loop

In [2910]:
# export
# Python native modules
import os,sys
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain,product
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules
from fastrl.core import test_in

IN_IPYTHON=False

_logger=logging.getLogger(__name__)

In [2911]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

### Why do we need this?
We have identified at least 3 different kinds of loops already:

    Learner (training)
    Source/Gym (Data Access)
    Agent (How an AI takes in data, generates actions)

### What is a loop?

    It should be capable of containing inner loops. 
    It should self-describe its structure. 
    It should be easy to know which parts of the loop are taking long/short amounts of time.
    It should be flexible in state modification.
    It should alternatively make it easy show what fields are being changed at what points in time.

A Loop will act as a compiled structure. The actual result will be a compiled list of nodes that reference the original loop.

In [2912]:
# export
class _Prop(object):
        
    def __getattr__(self,k): 
        self._attr=k
        return self
    def __call__(self,o): return getattr(o,self._attr)


class _PropCls(object):
    __metaclass__ = _Prop
        
    def __getattr__(self,k): return getattr(_Prop(),k)
    def __call__(self): return self

Prop=_PropCls()

In [2913]:
class TestA():
    @property
    def name(self): return self.__class__

class TestB():
    @property
    def name(self): return self.__class__

class TestC():
    @property
    def name(self): return self.__class__

L(TestA(),TestB(),TestC()).map(Prop.name)

(#3) [<class '__main__.TestA'>,<class '__main__.TestB'>,<class '__main__.TestC'>]

Compare that with what using `Self` does...

In [2914]:
test_fail(
    lambda:L(TestA(),TestB(),TestC()).map(Self.name)
)

In [2915]:
# export
EVENT_ORDER_MAPPING={}
PREFIXES=['before_','on_','after_','failed_','finally_']

def isevent(o): return issubclass(o.__class__,Event)
class EventException(Exception):pass
def _default_raise(_placeholder): raise 

class KwargSetAttr(object):
    def __setattr__(self,name,value):
        "Allow setting attrs via kwarg."
        super().__setattr__(name,value)

class Events(KwargSetAttr,L):
    def __init__(self,items=None,postfix=None,prefix=None,item_iter_hint='prefix',
                 order=0,parent_event=None,*args,**kwargs):
        store_attr(but='items')
        super().__init__(items=items,*args,**kwargs)
        
    def flat(self):
        return Events(chain.from_iterable(self),
                      postfix=self.postfix,prefix=self.prefix,
                      item_iter_hint=self.item_iter_hint,order=self.order,
                      parent_event=self.parent_event)

    def __lt__(self,o:'Event'): return self.order<o.order    
    def todict(self): 
        return {getattr(o,self.item_iter_hint):o for o in self}
    
    def __repr__(self): 
        if len(self)==0: return super().__repr__()
        return '\n'.join([str(o) for o in self])

class Event(KwargSetAttr):
    def __init__(self,
                 function:Callable,
                 loop=None,
                 parent_event=None,
                 override_name=None,
                 override_qualname=None,
                 override_module=None,
                 order=None
                ):
        store_attr()
        if self.function==noop and self.prefix=='failed_':
            self.function=_default_raise
        # We set the order over the entire Loop definition
        if self.order is None:
            if self.outer_name not in EVENT_ORDER_MAPPING: self.order=1
            else: self.order=EVENT_ORDER_MAPPING[self.outer_name]
            EVENT_ORDER_MAPPING[self.outer_name]=self.order+1

        self.original_name=self.function.__module__+'.'+self.function.__qualname__
            
        if self.name.startswith('_') or not any(self.name.startswith(pre) for pre in PREFIXES):
            raise EventException(f'{self.name} needs to start with any {PREFIXES}')
            
        self.cbs=L()
        
    def climb(self):
        "Returns a generator that moves up to the parent/root event"
        yield self
        if self.parent_event is not None:
            yield from self.parent_event.climb()
            
    @property
    def level(self):
        level=1
        for e in self.climb():
            level+=1
        return level
        
    @classmethod
    def from_override_name(cls,name,**kwargs):
        return cls(noop,override_name=name,**kwargs)
        
    def set_cbs(self,cbs=None):
        if cbs is not None:
            self.cbs=L((cb() if isinstance(cb, type) else cb) for cb in L(cbs) 
                       if hasattr(cb,self.name) and (not cb.call_on or any([isrelevent(cb,e) for e in self.climb()])))
            
    @property
    def root_loop(self): return self.loop.root_loop
    def __call__(self,*args,**kwargs): 
        ret=self.function(self.loop,*args,**kwargs)                             # fastrl.skip_traceback
        for cb in self.cbs: cb_ret=getattr(cb,self.name)()
        return ret

    def __lt__(self,o:'Event'): return self.order<o.order
    @property
    def name(self): return ifnone(self.override_name,self.function.__name__)
    @property
    def module(self): return ifnone(self.override_module,self.function.__module__)
    @property
    def qualname(self): return ifnone(self.override_qualname,self.function.__qualname__)
    @property
    def prefix(self): return self.name.split('_')[0]+'_'
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])
    @property
    def outer_name(self):
        return self.module+'.'+self.qualname.split('.')[0]
    
    def __repr__(self): return self.module+'.'+self.name
    def with_inner(self):
        return (self,Events(postfix=self.postfix,
                            prefix=self.prefix+'inner',
                            order=self.order))

event=Event

In [2916]:
Events(PREFIXES).map(Event.from_override_name).map(Prop.prefix)

before_
on_
after_
failed_
finally_

In [2917]:
def on_a():pass
decorated_a=Event(on_a)

def on_b():pass
decorated_b=Event(on_b,parent_event=decorated_a)

def on_c():pass
decorated_c=Event(on_c,parent_event=decorated_b)

Check that we can get the root event using the `climb` method...

In [2918]:
test_in(decorated_a,list(decorated_c.climb()))

Check that we can override the event's name...

In [2919]:
Event(on_c,parent_event=decorated_b,override_name='after_c')

__main__.after_c

In [2920]:
L(decorated_a,decorated_b,decorated_c).map(Prop.name)

(#3) ['on_a','on_b','on_c']

In [2921]:
L(decorated_a,decorated_b,decorated_c).map(Self.name)

(#3) [self: ['name']([(__main__.on_a,)], [{}]),'on_b','on_c']

In [2922]:
class B():
    @event
    def on_test_2_a(self):pass

In [2923]:
Events(L(B().on_test_2_a)).map(Event.__setattr__,name='parent_loop',value='thing')

None

In [2924]:
B.on_test_2_a.original_name

'__main__.B.on_test_2_a'

In [2925]:

class A(object):
    loop,root_loop=None,None
    
    @classmethod
    def events(cls,instance=None,instantiate=False):
        loop=instance if instance is not None else (cls() if instantiate else cls)
        events=L(inspect.getmembers(loop)).map(_last_element).filter(isevent)
        for o in events: o.loop=loop
        return events
    
    @event
    def before_test_c(self):print('before_test_c')
    @event
    def on_test_c(self):print('on_test_c')
    @event
    def after_test_c(self):print('after_test_c')
    @event
    def on_test_b(self):print('on_test_b')
    @event
    def on_test_a(self):print('on_test_a')

In [2926]:
A().before_test_c()

before_test_c


In [2927]:
A.events(A()).sorted()
test_eq(len(groupby(A.events(A()),Prop.postfix)),3)

Notes:
- Having a section object might make this overal loop management better
    - remaining stuff, the Section object needs to handle lists and execute those also.
    - with this in mind, using the section I think will make the looping run much cleaner.

In [2928]:
# export
def _show_events(e):
    return isevent(e) or (isinstance(e,Events) and len(e)!=0)

class SectionException(Exception):pass

class Section(KwargSetAttr):
    @with_cast
    def __init__(self,events:Events,parent_event=None):
        store_attr(but='events')
        # 1. Make Events have the same module as the function being run
        # 2. Convert the Events to Events+Inner Events
        # 3. Convert [(Event,[]*inner events*)...] to [Event,[]*inner events*...]
        # 4. Sure they are sorted correctly
        self.default_events=Events(PREFIXES)\
            .map(Event.from_override_name,override_module=events[0].module)\
            .map(Event.with_inner)\
            .flat()\
            .sorted()                                                           
        self.event_ls=events.sorted().map(Event.with_inner).flat()
        self.event_ls.map(Event.__setattr__,name='parent_event',value=parent_event)
        self.events=merge(self.default_events.todict(),self.event_ls.todict())
        
    def __repr__(self): 
        rep=str(self.__class__.__name__)+' '+self.postfix
        tab='\n'
        if self.parent_event is not None:
            tab= '\n'+'    '*self.parent_event.level
            rep+='\n'+'    '*self.parent_event.level
        else:
            rep+='\n'
        rep+=tab.join(L(self.events.values()).filter(_show_events).map(str))
        return rep
    
    @property
    def root_loop(self): return ifnone(self.event_ls[0].root_loop,self.loop)
    @property
    def loop(self):     return self.event_ls[0].loop
    @property
    def postfix(self):  return self.event_ls[0].postfix
    def __len__(self):  return len(L(self.events.values()).filter(isevent)) 
    def run_inner(self,event): [o.run() for o in self.events[event]]            # fastrl.skip_traceback
        
    def run(self):
        try:
            self.events['before_']()
            self.run_inner('before_inner')                                      # fastrl.skip_traceback
            self.events['on_']()
            self.run_inner('on_inner')
            self.events['after_']()
            self.run_inner('after_inner')                                       # fastrl.skip_traceback
        except Exception as ex:
            try:     
                self.events['failed_']()                                        # fastrl.skip_traceback
                raise
            finally: 
                self.run_inner('failed_inner')                                  # fastrl.skip_traceback
        finally:
            self.events['finally_']()
            self.run_inner('finally_inner')                                     # fastrl.skip_traceback
    
    @classmethod
    @with_cast
    def from_events(cls,events:Events,loop=None,parent_event=None):
        event_groups=groupby(events,Prop.postfix)
        return L(event_groups.values()).map(cls,parent_event=parent_event)
    
    def insert(self,loop,loops):
        "Insert a `loop` into the *inner events of a section"
        for event in self.event_ls.filter(isevent):
            if event.original_name in loop.call_on.map(Self.original_name()):
                loop=loop.from_loops(loops,event=event)
                self.events[event.prefix+'inner'].extend([loop])

In [2929]:
[o.events for o in Section.from_events(A.events(A()))]

[{'before_': __main__.before_test_c,
  'before_inner': [],
  'on_': __main__.on_test_c,
  'on_inner': [],
  'failed_': __main__.failed_,
  'failed_inner': [],
  'after_': __main__.after_test_c,
  'after_inner': [],
  'finally_': __main__.finally_,
  'finally_inner': []},
 {'failed_': __main__.failed_,
  'failed_inner': [],
  'before_': __main__.before_,
  'before_inner': [],
  'on_': __main__.on_test_a,
  'on_inner': [],
  'after_': __main__.after_,
  'after_inner': [],
  'finally_': __main__.finally_,
  'finally_inner': []},
 {'failed_': __main__.failed_,
  'failed_inner': [],
  'before_': __main__.before_,
  'before_inner': [],
  'on_': __main__.on_test_b,
  'on_inner': [],
  'after_': __main__.after_,
  'after_inner': [],
  'finally_': __main__.finally_,
  'finally_inner': []}]

In [2930]:
[o.run() for o in Section.from_events(A.events(A()))]

before_test_c
on_test_c
after_test_c
on_test_a
on_test_b


[None, None, None]

In [2931]:
# export
class class_or_instancemethod(classmethod):
    "From: https://stackoverflow.com/questions/28237955/same-name-for-classmethod-and-instancemethod"
    def __get__(self, instance, type_):
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, type_)

class Loop(object):  
    call_on,loop,loops,cbs,tab,verbose=L(),None,None,None,'  ',False
    
    @classmethod
    def sections(cls,loops:Optional[L]=None,event=None):
        events=L(inspect.getmembers(cls)).map(Self[-1]).filter(isevent).sorted()
        events.map(Event.__setattr__,name='loop',value=cls)
        sections=Section.from_events(events)
        if event is not None:
            events.map(Event.__setattr__,name='parent_event',value=event)
            sections.map(Section.__setattr__,name='parent_event',value=event)
        if loops is not None:
            for loop in loops:
                if isinstance(loop,cls) or loop==cls: continue
                for section in sections:
                    section.insert(loop,loops)
        return sections
    
    @classmethod
    def from_loops(cls,loops=None,event=None):
        loop=cls()
        sections=cls.sections(loops=loops,event=event)
        loop._sections=sections
        return loop
    
    def __repr__(self): 
        if hasattr(self,'_sections'):
            return str(self.__class__.__name__)+'\n'+'\n'.join(self._sections.map(str))
        return str(self.__class__.__name__)+' '
    
    def run(self,loops=None,cbs=None):
        try:
            sections=self.sections(loops=loops)                                 # fastrl.skip_traceback
            for section in sections:                                            # fastrl.skip_traceback
                section.run()                                                   # fastrl.skip_traceback
        except Exception as e:
            e._show_loop_errors=self.verbose
            raise
            
    @property
    def root_loop(self): return self if self.loop is None else self.loop
    def __len__(self):   return len(self.sections)

In [2932]:
class Outer(Loop):
    @event
    def before_step(self) :  print('before_step')
    @event
    def on_step(self)     :  print('on_step')
    @event
    def after_step(self)  :  print('after_step')
    @event
    def failed_step(self) :  print('failed_step')
    @event
    def finally_step(self):  print('finally_step')
 
    @event
    def before_jump(self) :  print('before_jump')
    @event
    def on_jump(self)     :  print('on_jump')
    @event
    def after_jump(self)  :  print('after_jump')
    @event
    def failed_jump(self) :  print('failed_jump')
    @event
    def finally_jump(self):  print('finally_jump')

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    @event
    def before_iteration(self) : print('before_iteration')
    @event
    def on_iteration(self)     : print('on_iteration')
    @event
    def after_iteration(self)  : print('after_iteration')
    @event
    def failed_iteration(self) : print('failed_iteration')
    @event
    def finally_iteration(self): print('finally_iteration')

class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    @event
    def on_force_fail(self):                    
        print('on_force_fail')
        raise Exception
        

In [2933]:
[o.run() for o in Outer.sections()]

before_step
on_step
after_step
finally_step
before_jump
on_jump
after_jump
finally_jump


[None, None]

In [2934]:
print(Outer.from_loops(L(Inner,FailingInner)))

Outer
Section step
__main__.failed_step
__main__.before_step
__main__.on_step
Inner
Section iteration
        __main__.failed_iteration
        __main__.before_iteration
        __main__.on_iteration
        __main__.after_iteration
        __main__.finally_iteration
        FailingInner
Section force_fail
            __main__.failed_
            __main__.before_
            __main__.on_force_fail
            __main__.after_
            __main__.finally_
__main__.after_step
Inner
Section iteration
        __main__.failed_iteration
        __main__.before_iteration
        __main__.on_iteration
        __main__.after_iteration
        __main__.finally_iteration
        FailingInner
Section force_fail
            __main__.failed_
            __main__.before_
            __main__.on_force_fail
            __main__.after_
            __main__.finally_
__main__.finally_step
Section jump
__main__.failed_jump
__main__.before_jump
__main__.on_jump
__main__.after_jump
__main__.finally_jump
Inne

In [2935]:
# export
class CallbackException(Exception):pass

class Callback(object):
    call_on,loop=None,None

In [2936]:
class OuterCallback(Callback):
    call_on=L(Outer.on_step)
    
    def before_iteration(self)->dict(this=list,that=str):
        print('   OuterCallback called lol')

Outer().get_sections(L(Inner(),FailingInner()),OuterCallback)

AttributeError: 'Outer' object has no attribute 'get_sections'

In [ ]:
print(Outer.from_sections(
    loops=L(Inner(),FailingInner()),
    cbs=OuterCallback
                   
                   
))

`Reference: https://stackoverflow.com/questions/31949760/how-to-limit-python-traceback-to-specific-files`

`Reference: https://github.com/ipython/ipython/blob/8520f3063ca36655b5febbbd18bf55e59cb2cbb5/IPython/core/interactiveshell.py#L1945`

In [ ]:
# export
def _skip_traceback(s):
    return in_('# fastrl.skip_traceback',s)
    
def ipy_handle_exception(self, etype, value, tb, tb_offset):
    ## Do something fancy
    stb = self.InteractiveTB.structured_traceback(etype,value,tb,tb_offset=tb_offset)
    if not getattr(value,'_show_loop_errors',True):
        tmp,idxs=[],L(stb).argwhere(_skip_traceback)
        prev_skipped_idx=idxs[0] if idxs else 0
        for i,s in enumerate(stb):
            if i in idxs and i-1!=prev_skipped_idx: 
                msg='Skipped Loop Code due to # fastrl.skip_traceback found in source code,'
                msg+=' please use Loop(...verbose=True) to view loop tracebacks\n'
                tmp.append(msg)
            if i not in idxs:
                tmp.append(s)
            else:
                prev_skipped_idx=i
        stb=tmp
    ## Do something fancy
    self._showtraceback(type, value, stb)

if IN_IPYTHON:
    get_ipython().set_custom_exc((Exception,),ipy_handle_exception)
    

In [ ]:
if False:
    Outer.verbose=False
    Outer().run(L(Inner(),FailingInner()),OuterCallback())

In [ ]:
if False:
    Outer.verbose=True
    Outer().run(L(Inner(),FailingInner()),OuterCallback())

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()
    # notebook2script('[!02_fastai.loop.old]*.ipynb')
    # notebook2html('[!02_fastai.loop.old]*.ipynb')

In [ ]:
# # export
# # Reference: https://stackoverflow.com/questions/31949760/how-to-limit-python-traceback-to-specific-files
# # __mycode = True

# # def is_mycode(tb):
# #     globals = tb.tb_frame.f_globals
# #     return globals.has_key('__mycode')

# # def mycode_traceback_levels(tb):
# #     length = 0
# #     while tb and is_mycode(tb):
# #         tb = tb.tb_next
# #         length += 1
# #     return length
# import traceback
# from traceback import TracebackException
# __mycode = True

# def callers_module():
#     module_name = inspect.currentframe().f_back.f_globals["__name__"]
#     return sys.modules[module_name]

# def is_mycode(tb):
#     return globals().get('__mycode',False)

# def mycode_traceback_levels(tb):
#     length = 0
#     while tb and is_mycode(tb):
#         tb = tb.tb_next
#         length += 1
#     return length

# def handle_exception(type, value, tb, tb_offset):
#     # 1. skip custom assert code, e.g.
#     # while tb and is_custom_assert_code(tb):
#     #   tb = tb.tb_next
#     # 2. only display your code
#     length = mycode_traceback_levels(tb)
    
#     # Reference: https://github.com/ipython/ipython/blob/8520f3063ca36655b5febbbd18bf55e59cb2cbb5/IPython/core/interactiveshell.py#L1945

#     # return [str(line) for line in TracebackException(
#     #         type, value, tb, limit=length).format(chain=True)]
#     return TracebackException(
#             type, value, tb, limit=length).format(chain=True)


# def ipy_handle_exception(self, etype, value, tb, tb_offset):
#     # 1. skip custom assert code, e.g.
#     # while tb and is_custom_assert_code(tb):
#     #   tb = tb.tb_next
#     # 2. only display your code
#     # length = mycode_traceback_levels(tb)
    
#     # Reference: https://github.com/ipython/ipython/blob/8520f3063ca36655b5febbbd18bf55e59cb2cbb5/IPython/core/interactiveshell.py#L1945

#     # return [str(line) for line in TracebackException(
#     #         type, value, tb, limit=length).format(chain=True)]
#     print('Returning traceback')
# #     ss= [str(line) for line in TracebackException(
# #             type, value, tb, limit=None).format(chain=True)]
    
#     stb = self.InteractiveTB.structured_traceback(etype,
#                                             value, tb, tb_offset=tb_offset)
    
#     # print(stb)
#     self._showtraceback(type, value, stb)
    
#     # print(ss)
#     # return ss
# get_ipython().set_custom_exc((Exception,),ipy_handle_exception)
    
# # sys.excepthook = handle_exception

# def custom_traceback():
#     exception_info=sys.exc_info()
#     # print(exception_info)
#     ex=handle_exception(*exception_info)
#     # get_ipython()
    